In [1]:
import os

In [2]:
%pwd

'c:\\Users\\asdaw\\Desktop\\Projects\\KidneyScan\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\asdaw\\Desktop\\Projects\\KidneyScan'

In [5]:
from pathlib import Path
from dataclasses import dataclass


@dataclass(frozen=True) 
class PrepareBaseModelConfig:
    root_dir: Path
    base_model_path: Path
    updated_base_model_path: Path
    params_image_size: list
    params_learning_rate: float 
    params_classes: int
    params_include_top: bool
    params_weights: str

In [6]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories

In [11]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_prepare_base_model_config(self) -> PrepareBaseModelConfig:
        config = self.config.prepare_base_model

        create_directories([config.root_dir])

        prepare_base_model_config = PrepareBaseModelConfig(
            root_dir=Path(config.root_dir),
            base_model_path=Path(config.base_model_path),
            updated_base_model_path=Path(config.updated_base_model_path),
            params_image_size=self.params.IMAGE_SIZE,
            params_learning_rate=self.params.LEARNING_RATE,
            params_classes=self.params.CLASSES,
            params_include_top=self.params.INCLUDE_TOP,
            params_weights=self.params.WEIGHTS
        )

        return prepare_base_model_config

 

In [12]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf

In [13]:
class PrepareBaseModel:
    def __init__(self, config: PrepareBaseModelConfig):
        self.config = config

    def get_base_model(self):
        self.model = tf.keras.applications.vgg16.VGG16(
            input_shape=self.config.params_image_size,
            weights=self.config.params_weights,
            include_top=self.config.params_include_top
        )
        self.save_model(path=self.config.base_model_path, model = self.model)
        
    @staticmethod
    def save_model(path:Path, model:tf.keras.Model):
        model.save(path)
        
    
    @staticmethod
    def _prepare_full_model(model, classes, freeze_all, freeze_till, learning_rate):
        if freeze_all:
            for layer in model.layers:
                layer.trainable = False
        elif (freeze_till is not None) and (freeze_till > 0):
            for layer in model.layers[:-freeze_till]:
                layer.trainable = False
        flatten_in = tf.keras.layers.Flatten()(model.output)
        predictions = tf.keras.layers.Dense(
            units=classes,
            activation="softmax"
        )(flatten_in)
        
        full_model = tf.keras.models.Model(
            inputs=model.input,
            outputs=predictions
        )
        
        full_model.compile(
            optimizer=tf.keras.optimizers.SGD(learning_rate=learning_rate),
            loss= tf.keras.losses.CategoricalCrossentropy(),
            metrics=["accuracy"]
        )
        
        full_model.summary()
        return full_model
    
    def update_base_model(self):
        self.full_model = self._prepare_full_model(
            model=self.model,
            classes=self.config.params_classes,
            freeze_all=True,
            freeze_till=None,
            learning_rate=self.config.params_learning_rate
        )
        self.save_model(path=self.config.updated_base_model_path, model = self.full_model)

In [14]:
try:
    config = ConfigurationManager()
    prepare_base_model_config = config.get_prepare_base_model_config() #it will  obtain the config for prepare base model

    prepare_base_model = PrepareBaseModel(config=prepare_base_model_config)  # it will create the object of PrepareBaseModel class
    prepare_base_model.get_base_model() 
    prepare_base_model.update_base_model()
except Exception as e:
    raise e

[2025-12-26 16:35:36,300: INFO: common]: yaml file: C:\Users\asdaw\Desktop\Projects\KidneyScan\config\config.yaml loaded successfully]
[2025-12-26 16:35:36,316: INFO: common]: yaml file: C:\Users\asdaw\Desktop\Projects\KidneyScan\params.yaml loaded successfully]
[2025-12-26 16:35:36,318: INFO: common]: created directory at: artificats]
[2025-12-26 16:35:36,321: INFO: common]: created directory at: artifacts/prepare_base_model]
58889256/58889256 [==============================] - 21s 0us/step
[2025-12-26 16:35:59,148: WARNING: saving_utils]: Compiled the loaded model, but the compiled metrics have yet to be built. `model.compile_metrics` will be empty until you train or evaluate the model.]
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)